In [1]:
import numpy as np
import matplotlib.pyplot as plt
import deepxde.deepxde as dde
import deepxde.deepxde.backend as bkd
from datasets import makeTesting
from datasets import parallel_solver, diffusion_reaction_solver
from utils.func import dirichlet, periodic
from utils.PDETriple import PDETripleCartesianProd

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [2]:
if False:
    makeTesting()

In [3]:
def pde(x, y, aux):
    D = 0.01
    k = 0.01
    dy_t = dde.grad.jacobian(y, x[1], j=1)
    dy_xx = dde.grad.hessian(y, x[1], j=0)
    out = dy_t - D * dy_xx + k * y**2 - aux
    return out

In [4]:
space = dde.data.GRF(1.0, length_scale = 0.1, N= 1000, interp="cubic")

geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)
vxs = space.eval_batch(space.random(1000), np.linspace(0, 1, 101)[:, None])
uxts = parallel_solver(diffusion_reaction_solver, vxs, num_workers = 6)
grid = uxts[0][0].reshape(101 * 101, -1)
uxts = np.asarray([u for grid, u in uxts]).reshape(-1, 101 * 101)

train_vxs = vxs
train_grid = grid
train_uxts = uxts
print(train_vxs.shape, train_grid.shape, train_uxts.shape)

test_data = np.load("datasets/DF_100_0.1_101_101.npz")
test_vxs = test_data["vxs"]
test_grid = test_data["xt"].reshape(-1, 2)
test_uxts = test_data["uxts"].reshape(-1, 101 * 101)
del test_data
print(test_vxs.shape, test_grid.shape, test_uxts.shape)

(20, 101) (10201, 2) (20, 10201)
(100, 101) (10201, 2) (100, 10201)


In [5]:
data = PDETripleCartesianProd(X_train=(train_vxs, train_grid), y_train=train_uxts, X_test=(test_vxs, test_grid), y_test=test_uxts)

# Net
net = dde.nn.DeepONetCartesianProd(
    [101, 128, 128, 128],
    [5, 128, 128, 128],
    "gelu",
    "Glorot normal",
)

net.apply_feature_transform(periodic)
net.apply_output_transform(dirichlet)

model = dde.Model(data, net)
model.compile("adam", lr=1E-3, loss= ["mse"], metrics = ["mean l2 relative error"], decay = ("inverse time", 200000, 0.5))


Compiling model...
'compile' took 0.000270 s



In [6]:
losshistory, train_state = model.train(iterations=20000 * 50, batch_size = len(train_vxs) // 10)
dde.utils.plot_loss_history(losshistory)

Training model...

Step      Train loss    Test loss     Test metric   
0         [1.08e-01]    [3.87e-01]    [1.49e+00]    
1000      [1.95e-02]    [8.71e-02]    [6.16e-01]    
2000      [1.20e-03]    [4.36e-02]    [4.85e-01]    
3000      [4.43e-04]    [3.81e-02]    [4.47e-01]    
4000      [1.32e-03]    [3.96e-02]    [4.53e-01]    
5000      [3.43e-02]    [1.09e-01]    [6.58e-01]    
6000      [1.09e-03]    [4.21e-02]    [4.66e-01]    
7000      [1.72e-03]    [4.56e-02]    [4.77e-01]    
